# Dogs vs Cats Classification - Kaggle

<br/>

##### Kaggle Link - [https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)

## All the imports needed.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from time import time
# tensorboard --logdir=logs/ --host localhost --port 8088

print(f'TensorFlow Version - {tf.__version__}')
print(f'Keras Version - {tf.keras.__version__}')

TensorFlow Version - 2.2.0
Keras Version - 2.3.0-tf


## Hyperparameters

In [2]:
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
IMG_SIZE = 50
LR = 0.0003
BATCH_SIZE = 32

MODEL_NAME = 'dogs_cats_LR-{}_MODEL-{}.h5'.format(LR,'CovNet-128(2)-64(2)-32(2)-512-128-1')
MODEL_PATH = os.path.join('saved_models',MODEL_NAME)

## Used model architecture for this project.

In [3]:
def get_model(saved=True):
    """This method returns the model used.
    Returns a saved model if MODEL_NAME is found.
    CovNet Architecture
    
    Keyword Arguments:
    saved - Get the saved model from the MODEL_PATH if exists.(default True)
    
    Returns:
    model - The complete uncompiled Keras model.
    """
    tf.reset_default_graph()
    
    if os.path.isfile(MODEL_PATH) and saved :
        print("Loading saved model {}".format(MODEL_NAME))
        return load_model(MODEL_PATH)
    
    # Declaring model
    model = Sequential()

    # 1st Block
    model.add(Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 1),filters=128, kernel_size=5, strides=1,padding='same',name = 'blk1_conv1'))
    model.add(Conv2D(filters=128, kernel_size=5, strides=1,padding='same',name = 'blk1_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk1_mxPool'))

    # 2nd Block
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'blk2_conv1'))
    model.add(Conv2D(filters=64, kernel_size=5, strides=1,padding='same',name = 'blk2_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk2_mxPool'))
    
    # 3rd Block
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv1'))
    model.add(Conv2D(filters=32, kernel_size=5, strides=1,padding='same',name = 'blk3_conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2,name = 'blk3_mxPool'))

    # 4th Block - FC Block
    dr_rate = 0.35
    model.add(Flatten(name = 'blk4_flatten'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout1'))
    model.add(Dense(512, activation='relu',name = 'blk4_dense1'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout2'))
    model.add(Dense(128, activation='relu',name = 'blk4_dense2'))
    model.add(Dropout(dr_rate,name = 'blk4_droupout3'))
    model.add(Dense(1, activation='sigmoid',name = 'blk4_dense3'))

    return model


## Utility Functions

In [4]:
def get_label(img):
    """Returns the label for an image.
    
    Keyword Arguments:
    img - The filename of the image whose label we want to get.
    
    Returns:
    list object - The respective label for dog or cat. (dog = 1, cat = 0)
    """
    word = img.split('.')[0]
    if word == 'cat':
        return [0]
    else:
        return [1]

In [5]:
def get_training_data():
    """Returns the training data from TRAIN_DIR.
    Images are read in grayscale format and resized to IMG_SIZE dimension square.
    The whole data is saved with numpy in .npy format for quick loading for future purpose.
    """
    training_data = []
    if os.path.isfile('training_data_{}.npy'.format(IMG_SIZE)):
        return np.load('training_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(TRAIN_DIR)):
            label = get_label(img)
            path = os.path.join(TRAIN_DIR,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            training_data.append([np.array(img),np.array(label)])
        shuffle(training_data)
        np.save('training_data_{}.npy'.format(IMG_SIZE),training_data)
        return np.array(training_data)

In [6]:
def get_testing_data():
    """Returns the testing data from TEST_DIR.
    Images are read in grayscale format and resized to IMG_SIZE dimension square.
    The whole data is saved with numpy in .npy format for quick loading for future purpose.
    """
    testing_data = []
    if os.path.isfile('testing_data_{}.npy'.format(IMG_SIZE)):
        return np.load('testing_data_{}.npy'.format(IMG_SIZE))
    else:
        for img in tqdm(os.listdir(TEST_DIR)):
            img_id = int(img.split('.')[0])
            path = os.path.join(TEST_DIR,img)
            img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE), (IMG_SIZE,IMG_SIZE))
            img = img/255
            testing_data.append([np.array(img),img_id])
        testing_data.sort(key = lambda x: x[1])
        np.save('testing_data_{}.npy'.format(IMG_SIZE),testing_data)
        return np.array(testing_data)

## Data Preparation

In [7]:
data = get_training_data()

partition = 1000             # Breaking -ve index
train = data[:-partition]    # For Training purpose
test= data[-partition:]      # For Validation purpose

# Training set
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = np.array([i[1] for i in train])

# Validation set
X_val = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_val = np.array([i[1] for i in test])

100%|██████████| 25000/25000 [06:45<00:00, 61.71it/s]


### Image Augmentation for better results.

In [8]:
"""Effects added on image
    Rotation - ± 50 deegrees,
    Width Shift - ± 15 %
    Height Shift - ± 15 %
    Zoom - 30%
    Horizontal Flip
    Vertical Flip
"""
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.05,height_shift_range=0.05,
                            zoom_range=0.05,horizontal_flip=True,vertical_flip=False)

# Calculation of necessary internal data for all images.
datagen.fit(X_train)

# Training Process

In [42]:
model = load_model('saved_models/dogs_cats_LR-0.0003_MODEL-CovNet-128(2)-64(2)-32(2)-512-128-1.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
blk1_conv1 (Conv2D)          (None, 50, 50, 128)       3328      
_________________________________________________________________
blk1_conv2 (Conv2D)          (None, 50, 50, 128)       409728    
_________________________________________________________________
blk1_mxPool (MaxPooling2D)   (None, 25, 25, 128)       0         
_________________________________________________________________
blk2_conv1 (Conv2D)          (None, 25, 25, 64)        204864    
_________________________________________________________________
blk2_conv2 (Conv2D)          (None, 25, 25, 64)        102464    
_________________________________________________________________
blk2_mxPool (MaxPooling2D)   (None, 12, 12, 64)        0         
_________________________________________________________________
blk3_conv1 (Conv2D)          (None, 12, 12, 32)        5

In [43]:
# Optimizer (Adam Optimizer)
adam = Adam(lr = LR)

# Callbacks Declared
tensorboard = TensorBoard(log_dir="logs/{}".format(time()),batch_size=BATCH_SIZE)
       #Supported in new version of keras ,update_freq='epoch')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.3,patience=3,verbose=1,
                              mode='max', min_lr=0.000001)
early_stop = EarlyStopping(monitor='val_loss',patience=3,verbose=1,mode='min')
      #Supported in new version of keras ,restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=MODEL_PATH,monitor='val_acc',verbose=1,save_best_only=True,
                                  mode='max',period=3)

model.compile(optimizer = adam,loss='binary_crossentropy',metrics=['accuracy'])

W0612 03:48:02.702359 140266105177920 callbacks.py:1747] `batch_size` is no longer needed in the `TensorBoard` Callback and will be ignored in TensorFlow 2.0.
W0612 03:48:03.141058 140266105177920 callbacks.py:1071] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of batches seen.


In [41]:
# Toggle if dont want to train using Image Augmentation
generator_train = True
EPOCHS = 30
callbacks=[tensorboard,reduce_lr,early_stop,model_checkpoint]

if generator_train:
    print(f'Training model {MODEL_NAME} using Image Augmentation')
    hist = model.fit_generator(datagen.flow(X_train,y_train,batch_size=BATCH_SIZE),
                               steps_per_epoch=len(X_train)//BATCH_SIZE,epochs=EPOCHS,verbose=2,
                               validation_data=(X_val,y_val),callbacks=callbacks)
else:
    print(f'Training model {MODEL_NAME} using normal image data provided')
    hist = model.fit(X_train,y_train,epochs=EPOCHS,batch_size=BATCH_SIZE,validation_data=(X_val,y_val),
                     verbose=2,callbacks=callbacks)
#model.save(MODEL_PATH)     Redundant Saving

Training model dogs_cats_LR-0.0003_MODEL-CovNet-128(2)-64(2)-32(2)-512-128-1.h5 using Image Augmentation


W0612 03:36:51.729134 140266105177920 deprecation.py:323] From <ipython-input-41-8e73921ed33a>:10: Model.fit_generator (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/30


KeyboardInterrupt: 

# Testing purpose

In [44]:
test_data = get_testing_data()

100%|██████████| 12500/12500 [03:42<00:00, 56.24it/s] 


In [46]:
X_test = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
ids = [i[1] for i in test_data]

In [47]:
pred = model.predict(X_test)

## Making the final submission CSV

In [48]:
filename = 'submission-{}.csv'.format(time())

with open(filename,'w') as f:
    f.write('id,label\n')
with open(filename,'a') as f:
    for i in range(len(X_test)):
        f.write('{},{}\n'.format(ids[i],pred[i][0]))

## acuracy testing

In [158]:
test_data.reshape(-1,1)
a=test_data[::2]
a.shape


(6250, 2)

In [146]:
ad = np.array([i[0] for i in pred]).reshape(-1,2)


In [159]:
ad.shape


(6250, 2)

In [109]:
from sklearn.metrics import accuracy_score

In [160]:
print(accuracy_score(ad,a))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()